<a href="https://colab.research.google.com/github/WannaGetDSJobInSanFrancisco/Machine_Notebooks/blob/main/ViLBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ViLBERT fine-tuning / evaluation for Hate Speech dataset
- Operation checked at: Colab PRO GPU mode or purchased large storage (>=approx 100GB)
  - Similar environment with Linux should be fine, but not tested

## Module Setup
- https://github.com/facebookresearch/vilbert-multi-task

### requirements.txt

In [ ]:
# !conda create -n vilbert-mt python=3.6
# !conda activate vilbert-mt
!git clone --recursive https://github.com/facebookresearch/vilbert-multi-task.git
%cd vilbert-multi-task
# https://stackoverflow.com/questions/22250483/stop-pip-from-failing-on-single-package-when-installing-with-requirements-txt
!cat requirements.txt | sed -e '/^\s*#.*$/d' -e '/^\s*$/d' | xargs -n 1 pip install

Cloning into 'vilbert-multi-task'...
remote: Enumerating objects: 2005, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2005 (delta 0), reused 0 (delta 0), pack-reused 2002
Receiving objects: 100% (2005/2005), 117.16 MiB | 22.03 MiB/s, done.
Resolving deltas: 100% (1436/1436), done.
Submodule 'tools/refer' (https://github.com/lichengunc/refer.git) registered for path 'tools/refer'
Cloning into '/content/vilbert-multi-task/tools/refer'...
remote: Enumerating objects: 185, done.        
remote: Counting objects: 100% (3/3), done.        
remote: Compressing objects: 100% (3/3), done.        
remote: Total 185 (delta 0), reused 0 (delta 0), pack-reused 182        
Receiving objects: 100% (185/185), 71.66 MiB | 32.70 MiB/s, done.
Resolving deltas: 100% (72/72), done.
Submodule path 'tools/refer': checked out 'd23ec5d8e84baf858c58af119799e26846aa5261'
/content/vilbert-multi-task
     |████████████████████████████████| 143kB 8.5M

### install the module failed in requirements.txt

In [ ]:
!apt-get -qq install -y python-prctl
# apt-get, not pip, worked for this module

Selecting previously unselected package python-prctl.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../python-prctl_1.6.1-2build1_amd64.deb ...
Unpacking python-prctl (1.6.1-2build1) ...
Setting up python-prctl (1.6.1-2build1) ...


### apex
- https://github.com/NVIDIA/apex

In [ ]:
# https://stackoverflow.com/questions/57284345/how-to-install-nvidia-apex-on-google-colab
!git clone https://github.com/NVIDIA/apex
%cd apex
# !pip install -v --disable-pip-version-check --no-cache-dir ./
# https://github.com/NVIDIA/apex/issues/161
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ../
%pwd

Cloning into 'apex'...
remote: Enumerating objects: 8038, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 8038 (delta 58), reused 69 (delta 30), pack-reused 7913
Receiving objects: 100% (8038/8038), 14.11 MiB | 19.49 MiB/s, done.
Resolving deltas: 100% (5457/5457), done.
/content/vilbert-multi-task/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-3bl3vd01
Created temporary directory: /tmp/pip-req-tracker-1t31vxio
Created requirements tracker '/tmp/pip-req-tracker-1t31vxio'
Created temporary directory: /tmp/pip-install-fdeexr5t
Processing /content/vilbert-multi-task/apex
  Created temporary directory: /tmp/pip-req-build-5a8l98kh
  Added file:///content/vilbert-multi-tas

'/content/vilbert-multi-task'

### setup.py

In [ ]:
!python setup.py develop

running develop
running egg_info
creating vilbert_multi_task.egg-info
writing vilbert_multi_task.egg-info/PKG-INFO
writing dependency_links to vilbert_multi_task.egg-info/dependency_links.txt
writing top-level names to vilbert_multi_task.egg-info/top_level.txt
writing manifest file 'vilbert_multi_task.egg-info/SOURCES.txt'
writing manifest file 'vilbert_multi_task.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/vilbert-multi-task.egg-link (link to .)
Adding vilbert-multi-task 0.1.0 to easy-install.pth file

Installed /content/vilbert-multi-task
Processing dependencies for vilbert-multi-task==0.1.0
Finished processing dependencies for vilbert-multi-task==0.1.0


### error handling

In [ ]:
# TypeError: Couldn't build proto file into descriptor pool!
## https://github.com/ValvePython/csgo/issues/8
!pip uninstall -y protobuf
!pip install --no-binary=protobuf protobuf

Uninstalling protobuf-3.12.4:
  Successfully uninstalled protobuf-3.12.4
     |████████████████████████████████| 235kB 8.1MB/s 
Skipping wheel build for protobuf, due to binaries being disabled for it.
    Running setup.py install for protobuf ... done


In [ ]:
# TypeError: Conflict register for file "tensorboard/compat/proto/tensor_shape.proto": tensorboard.TensorShapeProto is already defined in file "tensorboardX/src/tensor_shape.proto".
## xx
# !pip install allennlp
# !pip install allennlp-models
!pip install tensorboardX==1.8

     |████████████████████████████████| 225kB 8.7MB/s 
  Found existing installation: tensorboardX 1.2
    Uninstalling tensorboardX-1.2:
      Successfully uninstalled tensorboardX-1.2


In [ ]:
#   File "/content/vilbert-multi-task/tools/refer/refer.py", line 49
#    print 'loading dataset %s into memory...' % dataset                                          ^
# SyntaxError: Missing parentheses in call to 'print'. Did you mean print('loading dataset %s into memory...' % dataset)?
## So they're using python2 script :(
### https://github.com/facebookresearch/vilbert-multi-task/issues/33
%rm -r tools/refer
%mkdir tools/refer
!git clone -b python3 https://github.com/lichengunc/refer tools/refer
%cd tools/refer
!python setup.py install
!make
%cd ../
%cd ../
%pwd

Cloning into 'tools/refer'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 185 (delta 0), reused 0 (delta 0), pack-reused 182
Receiving objects: 100% (185/185), 71.66 MiB | 32.88 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/vilbert-multi-task/tools/refer
Compiling external/_mask.pyx because it changed.
[1/1] Cythonizing external/_mask.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/vilbert-multi-task/tools/refer/external/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running install
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/external
copying external/mask.py -> build/lib.linux-x86_64-3.7/external
copying external/__init__.py -> bu

'/content/vilbert-multi-task'

## download model
- https://stackoverflow.com/questions/49576657/is-there-anyway-i-can-download-the-file-in-google-colaboratory

In [ ]:
!wget https://dl.fbaipublicfiles.com/vilbert-multi-task/multi_task_model.bin

--2021-04-22 00:36:26--  https://dl.fbaipublicfiles.com/vilbert-multi-task/multi_task_model.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1072254755 (1023M) [application/octet-stream]
Saving to: ‘multi_task_model.bin’

multi_task_model.bi 100%[===================>]   1023M  59.7MB/s    in 16s     

2021-04-22 00:36:42 (62.8 MB/s) - ‘multi_task_model.bin’ saved [1072254755/1072254755]



## load data
- https://github.com/facebookresearch/vilbert-multi-task/tree/master/data

In [ ]:
# %cd data
!wget https://dl.fbaipublicfiles.com/vilbert-multi-task/datasets.tar.gz
# !tar tvf datasets.tar.gz
!tar xvf datasets.tar.gz $(tar tf datasets.tar.gz | grep 'VQA')
# !tar xvf datasets.tar.gz $(tar tf datasets.tar.gz | grep -i 'coco')
# !tar xf datasets.tar.gz --wildcards *coco
# !tar xf datasets.tar.gz

--2021-04-22 00:36:43--  https://dl.fbaipublicfiles.com/vilbert-multi-task/datasets.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97480948100 (91G) [application/gzip]
Saving to: ‘datasets.tar.gz’

datasets.tar.gz     100%[===================>]  90.79G  67.9MB/s    in 27m 48s 

2021-04-22 01:04:31 (55.7 MB/s) - ‘datasets.tar.gz’ saved [97480948100/97480948100]

datasets/VQA/
datasets/VQA/cache/
datasets/VQA/cache/train_target.pkl
datasets/VQA/cache/trainval_ans2label.pkl
datasets/VQA/cache/trainval_label2ans.pkl
datasets/VQA/cache/val_target.pkl
datasets/VQA/cache/coco_test_ids.npy
datasets/VQA/cache/VQA_trainval_23.pkl
datasets/VQA/cache/VQA_minval_23.pkl
datasets/VQA/cache/coco_test_ids_small.npy
datasets/VQA/cache/VQA_trainval_16.pkl
datasets/VQA/cache/VQA_minval_16.pkl

In [ ]:
# %cd data/
%rm datasets.tar.gz
%cd /content/vilbert-multi-task/

rm: cannot remove 'datasets.tar.gz': No such file or directory
/content/vilbert-multi-task


In [ ]:
# !mkdir datasets
%cd datasets
!mkdir coco
%cd coco
!mkdir features_100
%cd features_100
# !mkdir COCO_test_resnext152_faster_rcnn_genome.lmdb
!mkdir COCO_trainval_resnext152_faster_rcnn_genome.lmdb
!wget https://dl.fbaipublicfiles.com/vilbert-multi-task/datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb/data.mdb && mv data.mdb COCO_trainval_resnext152_faster_rcnn_genome.lmdb/
# !wget https://dl.fbaipublicfiles.com/vilbert-multi-task/datasets/coco/features_100/COCO_test_resnext152_faster_rcnn_genome.lmdb/data.mdb && mv data.mdb COCO_test_resnext152_faster_rcnn_genome.lmdb/
%cd /content/vilbert-multi-task/

/content/vilbert-multi-task/datasets
/content/vilbert-multi-task/datasets/coco
/content/vilbert-multi-task/datasets/coco/features_100
--2021-04-22 02:21:57--  https://dl.fbaipublicfiles.com/vilbert-multi-task/datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb/data.mdb
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101507162112 (95G) [application/octet-stream]
Saving to: ‘data.mdb’

data.mdb            100%[===================>]  94.54G  41.0MB/s    in 41m 3s  

2021-04-22 03:03:01 (39.3 MB/s) - ‘data.mdb’ saved [101507162112/101507162112]

/content/vilbert-multi-task


In [ ]:
# !mkdir datasets
# !mkdir datasets/coco
# !mkdir datasets/coco/features_100
# !mkdir datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb
# %mv features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb/data.mdb datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb

mv: cannot stat 'features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb/data.mdb': No such file or directory


In [ ]:
%cd /content/vilbert-multi-task/

/content/vilbert-multi-task


## Run train_tasks.py as fine tuning
- VQA as classification problem
  - https://github.com/facebookresearch/vilbert-multi-task/blob/master/vilbert_tasks.yml
  - https://arxiv.org/abs/1707.07998

In [ ]:
!python train_tasks.py --bert_model bert-base-uncased --from_pretrained multi_task_model.bin --config_file config/bert_base_6layer_6conect.json --tasks 1 --lr_scheduler 'warmup_linear' --train_iter_gap 4 --task_specific_tokens --save_name finetune_from_multi_task_model

2021-04-22 03:03:08.924205: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/22/2021 03:03:13 - INFO - vilbert.vilbert -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
04/22/2021 03:03:13 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
04/22/2021 03:03:14 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpynghkq4_
100% 231508/231508 [00:00<00:00, 963092.53B/s]
04/22/2021 03:03:14 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpynghkq4_ to cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/22/2021 03:03:14 - INFO - pytorch_transformers.file_utils -   creating metada

## fine tune

### init

In [ ]:
from types import SimpleNamespace

import torch
import numpy as np
from torchsummary import summary
from pytorch_transformers.tokenization_bert import BertTokenizer

from vilbert.vilbert import VILBertForVLTasks, BertConfig, BertForMultiModalPreTraining, BertImageEmbeddings

### modify path

In [ ]:
# PRETRAINED_MODEL_PATH = "multi_task_model.bin"
# !python train_tasks.py --bert_model bert-base-uncased --from_pretrained PRETRAINED_MODEL_PATH --config_file config/bert_base_6layer_6conect.json --tasks 1 --lr_scheduler 'warmup_linear' --train_iter_gap 4 --task_specific_tokens --save_name finetune_from_multi_task_model

In [ ]:
!mkdir save
!mkdir save/multitask_model
%mv multi_task_model.bin save/multitask_model/pytorch_model_9.bin

### load model

In [ ]:
args = SimpleNamespace(from_pretrained= "save/multitask_model/pytorch_model_9.bin",
                       bert_model="bert-base-uncased",
                       config_file="config/bert_base_6layer_6conect.json",
                       baseline=False,
                       num_labels = 3129, 
                       do_lower_case=True
                      )

In [ ]:
config = BertConfig.from_json_file(args.config_file)
if args.baseline:
    model = BaseBertForVLTasks.from_pretrained(
        args.from_pretrained, config=config, num_labels=args.num_labels, default_gpu=True
        )
else:
    model = VILBertForVLTasks.from_pretrained(
        args.from_pretrained, config=config, num_labels=args.num_labels, default_gpu=True
        )
cuda = torch.cuda.is_available()
if cuda: model = model.cuda(0)
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

100%|██████████| 231508/231508 [00:00<00:00, 688911.76B/s]


In [ ]:
class FeatureExtractor:
    MAX_SIZE = 1333
    MIN_SIZE = 800
    # MIN_SIZE = 10

    def __init__(self):
        self.args = self.get_parser()
        self.detection_model = self._build_detection_model()

    def get_parser(self):        
        parser = SimpleNamespace(model_file= 'save/resnext_models/model_final.pth',
                                #config_file='save/resnext_models/e2e_faster_rcnn_X-152-32x8d-FPN_1x_MLP_2048_FPN_512_train.yaml',
                                #config_file=os.path.join("save/resnext_models/", MODEL_NAME), 
                                config_file=os.path.join("save/resnext_models/model.yaml"), 
                                batch_size=1,
                                num_features=100,
                                feature_name="fc6",
                                confidence_threshold=0,
                                background=False,
                                partition=0)
        return parser
    
    def _build_detection_model(self):
        cfg.merge_from_file(self.args.config_file)
        cfg.freeze()

        model = build_detection_model(cfg)
        checkpoint = torch.load(self.args.model_file, map_location=torch.device("cpu"))

        load_state_dict(model, checkpoint.pop("model"))

        model.to("cuda")
        model.eval()
        return model

    def _image_transform(self, path):
        img = Image.open(path)
        im = np.array(img).astype(np.float32)
        # IndexError: too many indices for array, grayscale images
        if len(im.shape) < 3:
            im = np.repeat(im[:, :, np.newaxis], 3, axis=2)
        im = im[:, :, ::-1]
        im -= np.array([102.9801, 115.9465, 122.7717])
        im_shape = im.shape
        im_height = im_shape[0]
        im_width = im_shape[1]
        im_size_min = np.min(im_shape[0:2])
        im_size_max = np.max(im_shape[0:2])

        # Scale based on minimum size
        im_scale = self.MIN_SIZE / im_size_min

        # Prevent the biggest axis from being more than max_size
        # If bigger, scale it down
        if np.round(im_scale * im_size_max) > self.MAX_SIZE:
            im_scale = self.MAX_SIZE / im_size_max

        im = cv2.resize(
            im, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR
        )
        img = torch.from_numpy(im).permute(2, 0, 1)

        im_info = {"width": im_width, "height": im_height}

        return img, im_scale, im_info

    def _process_feature_extraction(
        self, output, im_scales, im_infos, feature_name="fc6", conf_thresh=0
    ):
        batch_size = len(output[0]["proposals"])
        n_boxes_per_image = [len(boxes) for boxes in output[0]["proposals"]]
        score_list = output[0]["scores"].split(n_boxes_per_image)
        score_list = [torch.nn.functional.softmax(x, -1) for x in score_list]
        feats = output[0][feature_name].split(n_boxes_per_image)
        cur_device = score_list[0].device

        feat_list = []
        info_list = []

        for i in range(batch_size):
            dets = output[0]["proposals"][i].bbox / im_scales[i]
            scores = score_list[i]
            max_conf = torch.zeros((scores.shape[0])).to(cur_device)
            conf_thresh_tensor = torch.full_like(max_conf, conf_thresh)
            start_index = 1
            # Column 0 of the scores matrix is for the background class
            if self.args.background:
                start_index = 0
            for cls_ind in range(start_index, scores.shape[1]):
                cls_scores = scores[:, cls_ind]
                keep = nms(dets, cls_scores, 0.5)
                max_conf[keep] = torch.where(
                    # Better than max one till now and minimally greater than conf_thresh
                    (cls_scores[keep] > max_conf[keep])
                    & (cls_scores[keep] > conf_thresh_tensor[keep]),
                    cls_scores[keep],
                    max_conf[keep],
                )

            sorted_scores, sorted_indices = torch.sort(max_conf, descending=True)
            num_boxes = (sorted_scores[: self.args.num_features] != 0).sum()
            keep_boxes = sorted_indices[: self.args.num_features]
            feat_list.append(feats[i][keep_boxes])
            bbox = output[0]["proposals"][i][keep_boxes].bbox / im_scales[i]
            # Predict the class label using the scores
            objects = torch.argmax(scores[keep_boxes][start_index:], dim=1)
            cls_prob = torch.max(scores[keep_boxes][start_index:], dim=1)

            info_list.append(
                {
                    "bbox": bbox.cpu().numpy(),
                    "num_boxes": num_boxes.item(),
                    "objects": objects.cpu().numpy(),
                    "image_width": im_infos[i]["width"],
                    "image_height": im_infos[i]["height"],
                    "cls_prob": scores[keep_boxes].cpu().numpy(),
                }
            )

        return feat_list, info_list

    def get_detectron_features(self, image_paths):
        img_tensor, im_scales, im_infos = [], [], []

        for image_path in image_paths:
            im, im_scale, im_info = self._image_transform(image_path)
            img_tensor.append(im)
            im_scales.append(im_scale)
            im_infos.append(im_info)

        # Image dimensions should be divisible by 32, to allow convolutions
        # in detector to work
        current_img_list = to_image_list(img_tensor, size_divisible=32)
        current_img_list = current_img_list.to("cuda")

        with torch.no_grad():
            output = self.detection_model(current_img_list)

        feat_list = self._process_feature_extraction(
            output,
            im_scales,
            im_infos,
            self.args.feature_name,
            self.args.confidence_threshold,
        )

        return feat_list

    def _chunks(self, array, chunk_size):
        for i in range(0, len(array), chunk_size):
            yield array[i : i + chunk_size]

    def _save_feature(self, file_name, feature, info):
        file_base_name = os.path.basename(file_name)
        file_base_name = file_base_name.split(".")[0]
        info["image_id"] = file_base_name
        info["features"] = feature.cpu().numpy()
        file_base_name = file_base_name + ".npy"

        np.save(os.path.join(self.args.output_folder, file_base_name), info)

    def extract_features(self, image_path):

        features, infos = self.get_detectron_features([image_path])

        return features, infos

In [ ]:
cuda = torch.cuda.is_available()
if cuda: model = model.cuda(0)
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

# 1: VQA, 2: GenomeQA, 4: Visual7w, 7: Retrieval COCO, 8: Retrieval Flickr30k 
# 9: refcoco, 10: refcoco+ 11: refcocog, 12: NLVR2, 13: VisualEntailment, 15: GQA, 16: GuessWhat, 


image_path = 'demo/1.jpg'
features, infos = feature_extractor.extract_features(image_path)

img = PIL.Image.open(image_path).convert('RGB')
img = torch.tensor(np.array(img))

plt.axis('off')
plt.imshow(img)
plt.show()
    
query = "swimming elephant"
task = [9]
custom_prediction(query, task, features, infos)

In [ ]:
model

In [ ]:
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (v_embeddings): BertImageEmbeddings(
      (image_embeddings): Linear(in_features=2048, out_features=1024, bias=True)
      (image_location_embeddings): Linear(in_features=5, out_features=1024, bias=True)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
          )
          (output): BertOutput(
            (dense): Linear(in_features=3072, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )

In [ ]:
tkn = ["she", "is", "beautiful"]
img = np.array([[1,2,3],[4,5,6]])
# img = np.array([1,2,3, 4,5,6])
# summary(model,(tkn, img, BertImageEmbeddings))
summary(model,(tkn, img, img))

TypeError: ignored

In [ ]:
summary(model,[500])

TypeError: ignored

## demo.ipynb

### additional setup

#### installation

In [ ]:
# first, make sure that your conda is setup properly with the right environment
# for that, check that `which conda`, `which pip` and `which python` points to the
# right path. From a clean conda env, this is what you need to do

# !conda create --name maskrcnn_benchmark -y
# !conda activate maskrcnn_benchmark

# this installs the right pip and dependencies for the fresh python
# !conda install ipython pip

# maskrcnn_benchmark and coco api dependencies
!pip install ninja yacs cython matplotlib tqdm opencv-python

# follow PyTorch installation in https://pytorch.org/get-started/locally/
# we give the instructions for CUDA 9.0
!conda install -c pytorch pytorch-nightly torchvision cudatoolkit=9.0

!export INSTALL_DIR=$PWD

# install pycocotools
%cd $INSTALL_DIR
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!python setup.py build_ext install

# install cityscapesScripts
%cd $INSTALL_DIR
!git clone https://github.com/mcordts/cityscapesScripts.git
%cd cityscapesScripts/
!python setup.py build_ext install

# # install apex
# %cd $INSTALL_DIR
# !git clone https://github.com/NVIDIA/apex.git
# %cd apex
# !python setup.py install --cuda_ext --cpp_ext

# # install PyTorch Detection
# %cd $INSTALL_DIR
# !git clone https://github.com/facebookresearch/maskrcnn-benchmark.git
# %cd maskrcnn-benchmark

# # the following will install the lib with
# # symbolic links, so that you can modify
# # the files if you want and won't need to
# # re-build it
# !python setup.py build develop

%cd /content/vilbert-multi-task

!unset INSTALL_DIR

# or if you are on macOS
# MACOSX_DEPLOYMENT_TARGET=10.9 CC=clang CXX=clang++ python setup.py build develop

#### error handling

In [ ]:
# FileNotFoundError: [Errno 2] No such file or directory: 'save/resnext_models/e2e_faster_rcnn_X-152-32x8d-FPN_1x_MLP_2048_FPN_512_train.yaml'
## https://github.com/facebookresearch/vilbert-multi-task/issues/6
%cd /content/vilbert-multi-task
!git clone https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark
%cd vqa-maskrcnn-benchmark
!python setup.py build develop
%cd ../

In [ ]:
# AttributeError: module 'torch._six' has no attribute 'PY3'
## https://datascience.stackexchange.com/questions/13669/how-to-export-one-cell-of-a-jupyter-notebook
# %%writefile maskrcnn-benchmark/maskrcnn_benchmark/utils/imports.py
%%writefile vqa-maskrcnn-benchmark/maskrcnn_benchmark/utils/imports.py
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.
import torch
dummy=False
# if torch._six.PY3:
if dummy:
    import importlib
    import importlib.util
    import sys


    # from https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    def import_file(module_name, file_path, make_importable=False):
        spec = importlib.util.spec_from_file_location(module_name, file_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        if make_importable:
            sys.modules[module_name] = module
        return module
else:
    import imp

    def import_file(module_name, file_path, make_importable=None):
        module = imp.load_source(module_name, file_path)
        return module

Overwriting vqa-maskrcnn-benchmark/maskrcnn_benchmark/utils/imports.py


In [ ]:
# move files to locations coressponding to demo.ipynb
## FileNotFoundError: [Errno 2] No such file or directory: 'save/resnext_models/e2e_faster_rcnn_X-152-32x8d-FPN_1x_MLP_2048_FPN_512_train.yaml'
# e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512_test.yaml
# e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512_train.yaml
# e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512_vqa_test.yaml
# e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512_vqa_val.yaml
# e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512_vqa_train.yaml
# e2e_faster_rcnn_X-152-32x8d-FPN_1x_MLP_2048_FPN_512_train.yaml 
# e2e_faster_rcnn_X-152-32x8d-FPN_1x_MLP_2048_FPN_512_test.yaml 
MODEL_NAME="e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512_train.yaml"
!mkdir save
!mkdir save/multitask_model
%mv multi_task_model.bin save/multitask_model/pytorch_model_9.bin
!mkdir save/resnext_models
# %mv vqa-maskrcnn-benchmark/configs/visual_genome_vqa/e2e_faster_rcnn_X-152-32x8d-FPN_1x_MLP_2048_FPN_512_train.yaml save/resnext_models/e2e_faster_rcnn_X-152-32x8d-FPN_1x_MLP_2048_FPN_512_train.yaml
%cd /content/vilbert-multi-task/vqa-maskrcnn-benchmark/configs/visual_genome_vqa/
%mv e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512_train.yaml /content/vilbert-multi-task/save/resnext_models/model.yaml
%cd /content/vilbert-multi-task

/content/vilbert-multi-task/vqa-maskrcnn-benchmark/configs/visual_genome_vqa
/content/vilbert-multi-task


In [ ]:
# no module named maskrcnn_benchmark
## https://stackoverflow.com/questions/57899205/modulenotfounderror-no-module-named-maskrcnn-benchmark
import sys
sys.path.append('/content/vilbert-multi-task/vqa-maskrcnn-benchmark')

### test
from maskrcnn_benchmark.utils.env import setup_environment  # noqa F401 isort:skip

In [ ]:
# FileNotFoundError: [Errno 2] No such file or directory: 'FAST_RCNN_MLP_DIM2048_FPN_DIM512.pkl'
## https://github.com/gabegrand/pythia-1/blob/master/data_prep/data_preprocess.md
## https://github.com/facebookresearch/mmf/issues/30
!wget https://dl.fbaipublicfiles.com/pythia/detectron_model/FAST_RCNN_MLP_DIM2048_FPN_DIM512.pkl

--2021-04-05 01:43:17--  https://dl.fbaipublicfiles.com/pythia/detectron_model/FAST_RCNN_MLP_DIM2048_FPN_DIM512.pkl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1357496266 (1.3G) [application/octet-stream]
Saving to: ‘FAST_RCNN_MLP_DIM2048_FPN_DIM512.pkl’

FAST_RCNN_MLP_DIM20 100%[===================>]   1.26G  11.7MB/s    in 2m 0s   

2021-04-05 01:45:18 (10.8 MB/s) - ‘FAST_RCNN_MLP_DIM2048_FPN_DIM512.pkl’ saved [1357496266/1357496266]



In [ ]:
%%writefile vqa-maskrcnn-benchmark/maskrcnn_benchmark/utils/c2_model_loading.py
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.
import logging
import pickle
from collections import OrderedDict

import torch

from maskrcnn_benchmark.utils.model_serialization import load_state_dict
from maskrcnn_benchmark.utils.registry import Registry


def _rename_basic_resnet_weights(layer_keys):
    layer_keys = [k.replace("_", ".") for k in layer_keys]
    layer_keys = [k.replace(".w", ".weight") for k in layer_keys]
    layer_keys = [k.replace(".bn", "_bn") for k in layer_keys]
    layer_keys = [k.replace(".b", ".bias") for k in layer_keys]
    layer_keys = [k.replace("_bn.s", "_bn.scale") for k in layer_keys]
    layer_keys = [k.replace(".biasranch", ".branch") for k in layer_keys]
    layer_keys = [k.replace("bbox.pred", "bbox_pred") for k in layer_keys]
    layer_keys = [k.replace("cls.score", "cls_score") for k in layer_keys]
    layer_keys = [k.replace("res.conv1_", "conv1_") for k in layer_keys]

    # RPN / Faster RCNN
    layer_keys = [k.replace(".biasbox", ".bbox") for k in layer_keys]
    layer_keys = [k.replace("conv.rpn", "rpn.conv") for k in layer_keys]
    layer_keys = [k.replace("rpn.bbox.pred", "rpn.bbox_pred") for k in layer_keys]
    layer_keys = [k.replace("rpn.cls.logits", "rpn.cls_logits") for k in layer_keys]

    # Affine-Channel -> BatchNorm enaming
    layer_keys = [k.replace("_bn.scale", "_bn.weight") for k in layer_keys]

    # Make torchvision-compatible
    layer_keys = [k.replace("conv1_bn.", "bn1.") for k in layer_keys]

    layer_keys = [k.replace("res2.", "layer1.") for k in layer_keys]
    layer_keys = [k.replace("res3.", "layer2.") for k in layer_keys]
    layer_keys = [k.replace("res4.", "layer3.") for k in layer_keys]
    layer_keys = [k.replace("res5.", "layer4.") for k in layer_keys]

    layer_keys = [k.replace(".branch2a.", ".conv1.") for k in layer_keys]
    layer_keys = [k.replace(".branch2a_bn.", ".bn1.") for k in layer_keys]
    layer_keys = [k.replace(".branch2b.", ".conv2.") for k in layer_keys]
    layer_keys = [k.replace(".branch2b_bn.", ".bn2.") for k in layer_keys]
    layer_keys = [k.replace(".branch2c.", ".conv3.") for k in layer_keys]
    layer_keys = [k.replace(".branch2c_bn.", ".bn3.") for k in layer_keys]

    layer_keys = [k.replace(".branch1.", ".downsample.0.") for k in layer_keys]
    layer_keys = [k.replace(".branch1_bn.", ".downsample.1.") for k in layer_keys]

    return layer_keys


def _rename_fpn_weights(layer_keys, stage_names):
    for mapped_idx, stage_name in enumerate(stage_names, 1):
        suffix = ""
        if mapped_idx < 4:
            suffix = ".lateral"
        layer_keys = [
            k.replace(
                "fpn.inner.layer{}.sum{}".format(stage_name, suffix),
                "fpn_inner{}".format(mapped_idx),
            )
            for k in layer_keys
        ]
        layer_keys = [
            k.replace("fpn.layer{}.sum".format(stage_name), "fpn_layer{}".format(mapped_idx))
            for k in layer_keys
        ]

    layer_keys = [k.replace("rpn.conv.fpn2", "rpn.conv") for k in layer_keys]
    layer_keys = [k.replace("rpn.bbox_pred.fpn2", "rpn.bbox_pred") for k in layer_keys]
    layer_keys = [k.replace("rpn.cls_logits.fpn2", "rpn.cls_logits") for k in layer_keys]

    return layer_keys


def _rename_weights_for_resnet(weights, stage_names):
    original_keys = sorted(weights.keys())
    layer_keys = sorted(weights.keys())

    # for X-101, rename output to fc1000 to avoid conflicts afterwards
    layer_keys = [k if k != "pred_b" else "fc1000_b" for k in layer_keys]
    layer_keys = [k if k != "pred_w" else "fc1000_w" for k in layer_keys]

    # performs basic renaming: _ -> . , etc
    layer_keys = _rename_basic_resnet_weights(layer_keys)

    # FPN
    layer_keys = _rename_fpn_weights(layer_keys, stage_names)

    # Mask R-CNN
    layer_keys = [k.replace("mask.fcn.logits", "mask_fcn_logits") for k in layer_keys]
    layer_keys = [k.replace(".[mask].fcn", "mask_fcn") for k in layer_keys]
    layer_keys = [k.replace("conv5.mask", "conv5_mask") for k in layer_keys]

    # Keypoint R-CNN
    layer_keys = [k.replace("kps.score.lowres", "kps_score_lowres") for k in layer_keys]
    layer_keys = [k.replace("kps.score", "kps_score") for k in layer_keys]
    layer_keys = [k.replace("conv.fcn", "conv_fcn") for k in layer_keys]

    # Rename for our RPN structure
    layer_keys = [k.replace("rpn.", "rpn.head.") for k in layer_keys]

    key_map = {k: v for k, v in zip(original_keys, layer_keys)}

    logger = logging.getLogger(__name__)
    logger.info("Remapping C2 weights")
    max_c2_key_size = max([len(k) for k in original_keys if "_momentum" not in k])

    new_weights = OrderedDict()
    for k in original_keys:
        v = weights[k]
        if "_momentum" in k:
            continue
        # if 'fc1000' in k:
        #     continue
        if v == "BGR":
            continue
        w = torch.from_numpy(v)
        # if "bn" in k:
        #     w = w.view(1, -1, 1, 1)
        logger.info("C2 name: {: <{}} mapped name: {}".format(k, max_c2_key_size, key_map[k]))
        new_weights[key_map[k]] = w

    return new_weights


def _load_c2_pickled_weights(file_path):
    with open(file_path, "rb") as f:
        dummy=True#False results in UnicodeDecodeError
        if dummy:
            data = pickle.load(f, encoding="latin1")
        else:
            data = pickle.load(f)
    if "blobs" in data:
        weights = data["blobs"]
    else:
        weights = data
    return weights


_C2_STAGE_NAMES = {
    "R-50": ["1.2", "2.3", "3.5", "4.2"], 
    "R-101": ["1.2", "2.3", "3.22", "4.2"], 
    "R-152": ["1.2", "2.7", "3.35", "4.2"]
}

C2_FORMAT_LOADER = Registry()


@C2_FORMAT_LOADER.register("R-50-C4")
@C2_FORMAT_LOADER.register("R-50-FPN")
@C2_FORMAT_LOADER.register("R-101-FPN")
@C2_FORMAT_LOADER.register("R-152-FPN")
def load_resnet_c2_format(cfg, f):
    state_dict = _load_c2_pickled_weights(f)
    conv_body = cfg.MODEL.BACKBONE.CONV_BODY
    arch = conv_body.replace("-C4", "").replace("-FPN", "")
    stages = _C2_STAGE_NAMES[arch]
    state_dict = _rename_weights_for_resnet(state_dict, stages)
    return dict(model=state_dict)


def load_c2_format(cfg, f):
    return C2_FORMAT_LOADER[cfg.MODEL.BACKBONE.CONV_BODY](cfg, f)


Overwriting vqa-maskrcnn-benchmark/maskrcnn_benchmark/utils/c2_model_loading.py


In [ ]:
# FileNotFoundError: [Errno 2] No such file or directory: 'save/resnext_models/model_final.pth'
## https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark/-/blob/master/tools/c2_to_pt.py
import os

import torch as t
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.utils.c2_model_loading import load_c2_format

path = "FAST_RCNN_MLP_DIM2048_FPN_DIM512.pkl"
config_name = 'modified_for_conversion_e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512.yaml'
base_path = '/content/vilbert-multi-task/vqa-maskrcnn-benchmark/configs/visual_genome_vqa/c2'
# base_path = '/content/vilbert-multi-task/save/visual_genome_vqa/c2'

cfg.merge_from_file(os.path.join(base_path, config_name))

_d = load_c2_format(cfg, path)
newdict = _d
t.save(newdict, "save/resnext_models/model_final.pth")

/content/vilbert-multi-task/vqa-maskrcnn-benchmark/maskrcnn_benchmark/utils/c2_model_loading.py:117: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if v == "BGR":


#### run notebook

In [ ]:
import sys
import os
import torch
import yaml

from easydict import EasyDict as edict
from pytorch_transformers.tokenization_bert import BertTokenizer
from vilbert.datasets import ConceptCapLoaderTrain, ConceptCapLoaderVal
from vilbert.vilbert import VILBertForVLTasks, BertConfig, BertForMultiModalPreTraining
from vilbert.task_utils import LoadDatasetEval

import numpy as np
import matplotlib.pyplot as plt
import PIL

from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.layers import nms
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.structures.image_list import to_image_list
from maskrcnn_benchmark.utils.model_serialization import load_state_dict
from PIL import Image
import cv2
import argparse
import glob
from types import SimpleNamespace
import pdb

%matplotlib inline  

04/05/2021 01:45:25 - INFO - vilbert.vilbert -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
class FeatureExtractor:
    MAX_SIZE = 1333
    # MIN_SIZE = 800
    MIN_SIZE = 10

    def __init__(self):
        self.args = self.get_parser()
        self.detection_model = self._build_detection_model()

    def get_parser(self):        
        parser = SimpleNamespace(model_file= 'save/resnext_models/model_final.pth',
                                #config_file='save/resnext_models/e2e_faster_rcnn_X-152-32x8d-FPN_1x_MLP_2048_FPN_512_train.yaml',
                                #config_file=os.path.join("save/resnext_models/", MODEL_NAME), 
                                config_file=os.path.join("save/resnext_models/model.yaml"), 
                                batch_size=1,
                                num_features=100,
                                feature_name="fc6",
                                confidence_threshold=0,
                                background=False,
                                partition=0)
        return parser
    
    def _build_detection_model(self):
        cfg.merge_from_file(self.args.config_file)
        cfg.freeze()

        model = build_detection_model(cfg)
        checkpoint = torch.load(self.args.model_file, map_location=torch.device("cpu"))

        load_state_dict(model, checkpoint.pop("model"))

        model.to("cuda")
        model.eval()
        return model

    def _image_transform(self, path):
        img = Image.open(path)
        im = np.array(img).astype(np.float32)
        # IndexError: too many indices for array, grayscale images
        if len(im.shape) < 3:
            im = np.repeat(im[:, :, np.newaxis], 3, axis=2)
        im = im[:, :, ::-1]
        im -= np.array([102.9801, 115.9465, 122.7717])
        im_shape = im.shape
        im_height = im_shape[0]
        im_width = im_shape[1]
        im_size_min = np.min(im_shape[0:2])
        im_size_max = np.max(im_shape[0:2])

        # Scale based on minimum size
        im_scale = self.MIN_SIZE / im_size_min

        # Prevent the biggest axis from being more than max_size
        # If bigger, scale it down
        if np.round(im_scale * im_size_max) > self.MAX_SIZE:
            im_scale = self.MAX_SIZE / im_size_max

        im = cv2.resize(
            im, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR
        )
        img = torch.from_numpy(im).permute(2, 0, 1)

        im_info = {"width": im_width, "height": im_height}

        return img, im_scale, im_info

    def _process_feature_extraction(
        self, output, im_scales, im_infos, feature_name="fc6", conf_thresh=0
    ):
        batch_size = len(output[0]["proposals"])
        n_boxes_per_image = [len(boxes) for boxes in output[0]["proposals"]]
        score_list = output[0]["scores"].split(n_boxes_per_image)
        score_list = [torch.nn.functional.softmax(x, -1) for x in score_list]
        feats = output[0][feature_name].split(n_boxes_per_image)
        cur_device = score_list[0].device

        feat_list = []
        info_list = []

        for i in range(batch_size):
            dets = output[0]["proposals"][i].bbox / im_scales[i]
            scores = score_list[i]
            max_conf = torch.zeros((scores.shape[0])).to(cur_device)
            conf_thresh_tensor = torch.full_like(max_conf, conf_thresh)
            start_index = 1
            # Column 0 of the scores matrix is for the background class
            if self.args.background:
                start_index = 0
            for cls_ind in range(start_index, scores.shape[1]):
                cls_scores = scores[:, cls_ind]
                keep = nms(dets, cls_scores, 0.5)
                max_conf[keep] = torch.where(
                    # Better than max one till now and minimally greater than conf_thresh
                    (cls_scores[keep] > max_conf[keep])
                    & (cls_scores[keep] > conf_thresh_tensor[keep]),
                    cls_scores[keep],
                    max_conf[keep],
                )

            sorted_scores, sorted_indices = torch.sort(max_conf, descending=True)
            num_boxes = (sorted_scores[: self.args.num_features] != 0).sum()
            keep_boxes = sorted_indices[: self.args.num_features]
            feat_list.append(feats[i][keep_boxes])
            bbox = output[0]["proposals"][i][keep_boxes].bbox / im_scales[i]
            # Predict the class label using the scores
            objects = torch.argmax(scores[keep_boxes][start_index:], dim=1)
            cls_prob = torch.max(scores[keep_boxes][start_index:], dim=1)

            info_list.append(
                {
                    "bbox": bbox.cpu().numpy(),
                    "num_boxes": num_boxes.item(),
                    "objects": objects.cpu().numpy(),
                    "image_width": im_infos[i]["width"],
                    "image_height": im_infos[i]["height"],
                    "cls_prob": scores[keep_boxes].cpu().numpy(),
                }
            )

        return feat_list, info_list

    def get_detectron_features(self, image_paths):
        img_tensor, im_scales, im_infos = [], [], []

        for image_path in image_paths:
            im, im_scale, im_info = self._image_transform(image_path)
            img_tensor.append(im)
            im_scales.append(im_scale)
            im_infos.append(im_info)

        # Image dimensions should be divisible by 32, to allow convolutions
        # in detector to work
        current_img_list = to_image_list(img_tensor, size_divisible=32)
        current_img_list = current_img_list.to("cuda")

        with torch.no_grad():
            output = self.detection_model(current_img_list)

        feat_list = self._process_feature_extraction(
            output,
            im_scales,
            im_infos,
            self.args.feature_name,
            self.args.confidence_threshold,
        )

        return feat_list

    def _chunks(self, array, chunk_size):
        for i in range(0, len(array), chunk_size):
            yield array[i : i + chunk_size]

    def _save_feature(self, file_name, feature, info):
        file_base_name = os.path.basename(file_name)
        file_base_name = file_base_name.split(".")[0]
        info["image_id"] = file_base_name
        info["features"] = feature.cpu().numpy()
        file_base_name = file_base_name + ".npy"

        np.save(os.path.join(self.args.output_folder, file_base_name), info)

    def extract_features(self, image_path):

        features, infos = self.get_detectron_features([image_path])

        return features, infos


In [ ]:
def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

def printer(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)[1:-1]
    print(" ".join(sent))


# write arbitary string for given sentense. 
import _pickle as cPickle

In [ ]:
def prediction(question, features, spatials, segment_ids, input_mask, image_mask, co_attention_mask, task_tokens, ):

    vil_prediction, vil_prediction_gqa, vil_logit, vil_binary_prediction, vil_tri_prediction, vision_prediction, vision_logit, linguisic_prediction, linguisic_logit, attn_data_list = model(
        question, features, spatials, segment_ids, input_mask, image_mask, co_attention_mask, task_tokens, output_all_attention_masks=True
    )
    
    height, width = img.shape[0], img.shape[1]

    logits = torch.max(vil_prediction, 1)[1].data  # argmax
    # Load VQA label to answers:
    label2ans_path = os.path.join('save', "VQA" ,"cache", "trainval_label2ans.pkl")
    vqa_label2ans = cPickle.load(open(label2ans_path, "rb"))
    answer = vqa_label2ans[logits[0].item()]
    print("VQA: " + answer)

    # Load GQA label to answers:
    label2ans_path = os.path.join('save', "gqa" ,"cache", "trainval_label2ans.pkl")

    logtis_gqa = torch.max(vil_prediction_gqa, 1)[1].data
    gqa_label2ans = cPickle.load(open(label2ans_path, "rb"))
    answer = gqa_label2ans[logtis_gqa[0].item()]
    print("GQA: " + answer)

    # vil_binary_prediction NLVR2, 0: False 1: True Task 12
    logtis_binary = torch.max(vil_binary_prediction, 1)[1].data
    print("NLVR: " + str(logtis_binary.item()))

    # vil_entaliment:  
    label_map = {0:"contradiction", 1:"neutral", 2:"entailment"}
    logtis_tri = torch.max(vil_tri_prediction, 1)[1].data
    print("Entaliment: " + str(label_map[logtis_tri.item()]))

    # vil_logit: 
    logits_vil = vil_logit[0].item()
    print("ViL_logit: %f" %logits_vil)

    # grounding: 
    logits_vision = torch.max(vision_logit, 1)[1].data
    grounding_val, grounding_idx = torch.sort(vision_logit.view(-1), 0, True)

    examples_per_row = 5
    ncols = examples_per_row 
    nrows = 1
    figsize = [12, ncols*20]     # figure size, inches
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)

    for i, axi in enumerate(ax.flat):
        idx = grounding_idx[i]
        val = grounding_val[i]
        box = spatials[0][idx][:4].tolist()
        y1 = int(box[1] * height)
        y2 = int(box[3] * height)
        x1 = int(box[0] * width)
        x2 = int(box[2] * width)
        patch = img[y1:y2,x1:x2]
        axi.imshow(patch)
        axi.axis('off')
        axi.set_title(str(i) + ": " + str(val.item()))

    plt.axis('off')
    plt.tight_layout(True)
    plt.show()  

In [ ]:
def custom_prediction(query, task, features, infos):

    tokens = tokenizer.encode(query)
    tokens = tokenizer.add_special_tokens_single_sentence(tokens)

    segment_ids = [0] * len(tokens)
    input_mask = [1] * len(tokens)

    max_length = 37
    if len(tokens) < max_length:
        # Note here we pad in front of the sentence
        padding = [0] * (max_length - len(tokens))
        tokens = tokens + padding
        input_mask += padding
        segment_ids += padding

    text = torch.from_numpy(np.array(tokens)).cuda().unsqueeze(0)
    input_mask = torch.from_numpy(np.array(input_mask)).cuda().unsqueeze(0)
    segment_ids = torch.from_numpy(np.array(segment_ids)).cuda().unsqueeze(0)
    task = torch.from_numpy(np.array(task)).cuda().unsqueeze(0)

    num_image = len(infos)

    feature_list = []
    image_location_list = []
    image_mask_list = []
    for i in range(num_image):
        image_w = infos[i]['image_width']
        image_h = infos[i]['image_height']
        feature = features[i]
        num_boxes = feature.shape[0]

        g_feat = torch.sum(feature, dim=0) / num_boxes
        num_boxes = num_boxes + 1
        feature = torch.cat([g_feat.view(1,-1), feature], dim=0)
        boxes = infos[i]['bbox']
        image_location = np.zeros((boxes.shape[0], 5), dtype=np.float32)
        image_location[:,:4] = boxes
        image_location[:,4] = (image_location[:,3] - image_location[:,1]) * (image_location[:,2] - image_location[:,0]) / (float(image_w) * float(image_h))
        image_location[:,0] = image_location[:,0] / float(image_w)
        image_location[:,1] = image_location[:,1] / float(image_h)
        image_location[:,2] = image_location[:,2] / float(image_w)
        image_location[:,3] = image_location[:,3] / float(image_h)
        g_location = np.array([0,0,1,1,1])
        image_location = np.concatenate([np.expand_dims(g_location, axis=0), image_location], axis=0)
        image_mask = [1] * (int(num_boxes))

        feature_list.append(feature)
        image_location_list.append(torch.tensor(image_location))
        image_mask_list.append(torch.tensor(image_mask))

    features = torch.stack(feature_list, dim=0).float().cuda()
    spatials = torch.stack(image_location_list, dim=0).float().cuda()
    image_mask = torch.stack(image_mask_list, dim=0).byte().cuda()
    co_attention_mask = torch.zeros((num_image, num_boxes, max_length)).cuda()

    prediction(text, features, spatials, segment_ids, input_mask, image_mask, co_attention_mask, task)

In [ ]:

# =============================
# ViLBERT part
# =============================
feature_extractor = FeatureExtractor()

args = SimpleNamespace(from_pretrained= "save/multitask_model/pytorch_model_9.bin",
                       bert_model="bert-base-uncased",
                       config_file="config/bert_base_6layer_6conect.json",
                       max_seq_length=101,
                       train_batch_size=1,
                       do_lower_case=True,
                       predict_feature=False,
                       seed=42,
                       num_workers=0,
                       baseline=False,
                       img_weight=1,
                       distributed=False,
                       objective=1,
                       visual_target=0,
                       dynamic_attention=False,
                       task_specific_tokens=True,
                       tasks='1',
                       save_name='',
                       in_memory=False,
                       batch_size=1,
                       local_rank=-1,
                       split='mteval',
                       clean_train_sets=True
                      )

config = BertConfig.from_json_file(args.config_file)
with open('./vilbert_tasks.yml', 'r') as f:
    task_cfg = edict(yaml.safe_load(f))

task_names = []
for i, task_id in enumerate(args.tasks.split('-')):
    task = 'TASK' + task_id
    name = task_cfg[task]['name']
    task_names.append(name)

timeStamp = args.from_pretrained.split('/')[-1] + '-' + args.save_name
config = BertConfig.from_json_file(args.config_file)
default_gpu=True

if args.predict_feature:
    config.v_target_size = 2048
    config.predict_feature = True
else:
    config.v_target_size = 1601
    config.predict_feature = False

if args.task_specific_tokens:
    config.task_specific_tokens = True    

if args.dynamic_attention:
    config.dynamic_attention = True

config.visualization = True
num_labels = 3129

if args.baseline:
    model = BaseBertForVLTasks.from_pretrained(
        args.from_pretrained, config=config, num_labels=num_labels, default_gpu=default_gpu
        )
else:
    model = VILBertForVLTasks.from_pretrained(
        args.from_pretrained, config=config, num_labels=num_labels, default_gpu=default_gpu
        )
    
model.eval()
cuda = torch.cuda.is_available()
if cuda: model = model.cuda(0)
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)


In [ ]:
# 1: VQA, 2: GenomeQA, 4: Visual7w, 7: Retrieval COCO, 8: Retrieval Flickr30k 
# 9: refcoco, 10: refcoco+ 11: refcocog, 12: NLVR2, 13: VisualEntailment, 15: GQA, 16: GuessWhat, 


image_path = 'demo/1.jpg'
features, infos = feature_extractor.extract_features(image_path)

img = PIL.Image.open(image_path).convert('RGB')
img = torch.tensor(np.array(img))

plt.axis('off')
plt.imshow(img)
plt.show()
    
query = "swimming elephant"
task = [9]
custom_prediction(query, task, features, infos)

KeyError: ignored

## summary
- one issue remaining, however it seems is raised from another repository, therefore just ignore it for now and proceed
  - https://github.com/facebookresearch/vilbert-multi-task/issues/90